In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm
import matplotlib as mpl
import numpy as np
from h5py import File
import napari
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm
import matplotlib as mpl
import numpy as np
import napari
import colorcet as cc

spots_directory = Path(r"D:\Tracking\DrosophilaNucleusTracking\data\spots")
save_path = Path(r"D:\Tracking\DrosophilaNucleusTracking\figures\output")

rc = {
    # "figure.facecolor": "#FFFFFF00",
    # "axes.facecolor": "#FFFFFF00",
    "legend.framealpha": 0.2,
    "lines.color": "k",
    "svg.fonttype": "none",
}

sns.set_context("paper")
sns.set_theme(style="ticks", font="Arial", rc=rc)
# print(mpl.font_manager.get_font_names())
nc_pal = {11: "coral", 12: "mediumseagreen", 13: "cadetblue", 14: "#895794"}

trk_pal = ["steelblue", "gold"]
four_color_pal = ["#44af69", "#f8333c", "#fcab10", "#2b9eb3"]
spots_dfs = []
metadatas = []
stems = []

directories = list(spots_directory.glob("*_spots.h5"))
print([f"{d.stem}" for d in directories])

for i, spots_path in tqdm(enumerate(spots_directory.glob("*_spots.h5")), desc="reading spots dfs"):
    spots_df = pd.read_hdf(spots_path, key="df")
    stems.append(spots_path.stem)
    spots_df["source"] = i

    metadata = {}

    with File(spots_path, "r") as f:
        m = f["metadata"]
        metadata.update(m.attrs)

    metadatas.append(metadata)

    # print(metadata)

    dtw_path = spots_directory / "dtw" / f"{spots_path.stem}_dtw.h5"
    if dtw_path.exists():
        dtw_update = pd.read_hdf(dtw_path, key="df")
        spots_df = pd.merge(
            spots_df, dtw_update, left_index=True, right_index=True, how="left"
        )
        spots_df["cycle_pseudotime"] = spots_df["pseudotime"] + spots_df["cycle"] - 11
    else:
        spots_df["distance"] = np.nan  # rename this later
        spots_df["pseudotime"] = np.nan

    spots_dfs.append(spots_df)

all_dfs = pd.concat(spots_dfs, ignore_index=True)

['lightsheet_20241104_spots', 'lightsheet_20250131_spots', 'lightsheet_20250206_spots', 'lightsheet_20250414_spots', 'lightsheet_20250625_spots', 'lightsheet_20250627_spots', 'lightsheet_20250702_spots', 'lightsheet_20250704_spots', 'lightsheet_20250705_spots', 'lightsheet_trk_20250318_spots', 'lightsheet_trk_20250319a_spots', 'lightsheet_trk_20250319b_spots', 'lightsheet_trk_20250321_spots', 'lightsheet_trk_20250325_spots']


reading spots dfs: 14it [00:09,  1.42it/s]


In [19]:
from sklearn.neighbors import NearestNeighbors

def load_frame_data(df, neighbors=5):

    df = df.copy()
    df["true_value"] = 0
    df["fit_value"] = 0

    for frame in tqdm(df["frame"].unique()):
        frame_subset = df[df["frame"] == frame].copy()
        # values = np.array([frame - mid for mid in frame_subset["cycle_start"]])
        # values[values > 12] = -1
        # values[(values >= 0) & (values <= 1)] = 0
        # values[(values <= 12) & (values > 1)] = 1
        values = frame_subset["cycle"].values
        spatial = frame_subset[["x", "y", "z"]].values

        knn = NearestNeighbors(n_neighbors=neighbors)
        knn.fit(spatial, values)
        fit_values = values[knn.kneighbors(spatial, return_distance=False)]
        fit_values = np.array(np.median(fit_values, axis=1), dtype=int)

        ii = frame_subset.index

        df.loc[ii, "true_value"] = values
        df.loc[ii, "fit_value"] = fit_values

    return df

i = 3

spots_df = spots_dfs[i]
spots_df["cycle_start"] = spots_df["tracklet_id"].map(spots_df.groupby("tracklet_id")["frame"].min())

# spots_df = load_frame_data(spots_df)

In [23]:
stem = stems[i]
mako = sns.color_palette("mako", as_cmap=True)
nc_pal = {10: mako(0.2), 11: mako(0.3), 12: mako(0.4), 13:mako(0.5), 14: mako(0.6)}
nc_pal = {cycle: col for cycle, col in zip(
    [9, 10, 11, 12, 13, 14],
    ["#555555", "#555555", "#44af69", "#f8333c", "#fcab10", "#2b9eb3"]
)}

viewer = napari.Viewer(ndisplay=3)

X = spots_df[["frame", "display_x", "display_y", "display_z"]].values
size = spots_df["radius"] * 2

color = [nc_pal[val] for val in spots_df["cycle"]]

from napari.utils.theme import get_theme, register_theme
custom_theme = get_theme("dark")
custom_theme.canvas = "#ffffff"
custom_theme.id = "glasbey"
register_theme("glasbey", custom_theme, "custom")
viewer.theme = "glasbey"

points = viewer.add_points(
    X, face_color=color, size=size, name=f"spots_{i}", border_width=1.0, border_width_is_relative=False,
    antialiasing=1.0, border_color="#000000",
)

viewer.camera.angles = (0, 180, 90)
viewer.camera.zoom = 1.0

In [4]:
wave_save_path = save_path / "wave_horizontal" / stem
wave_save_path.mkdir(parents=True, exist_ok=True)

for frame in tqdm(spots_df["frame"].unique()):
    viewer.dims.set_point(0, frame)
    viewer.screenshot(wave_save_path / f"{frame:04d}.tif", canvas_only=True, scale=1, flash=False)

100%|██████████| 248/248 [00:12<00:00, 20.28it/s]


In [15]:
i = 1

spots_df = spots_dfs[i]

stem = stems[i]

mesh_pal = {"poles": "red", "top": "green", "bottom": "blue"}

viewer = napari.Viewer(ndisplay=3)
viewer.theme = "glasbey"

spots_df = spots_df[spots_df["z"] > spots_df["z"].mean()]

X = spots_df[["frame", "x", "y", "z"]].values * 0.25
size = spots_df["radius"] * 0.5

color = [mesh_pal[val] for val in spots_df["mesh_name"]]

points = viewer.add_points(
    X, face_color=color, size=size, name=f"spots_{i}", border_width=0.25, border_width_is_relative=False,
    antialiasing=1.0, border_color="#000000",
)

viewer.camera.angles = (0, 180, 90)
viewer.camera.zoom = 1.0